# Testing melting and solidification
- Considering a box with width 100 km and height 100 km.
- The box is filled entirely with one material
- The sides are insulating
- Density is constant and equal to 1000 kg/m$^3$
- The surface is at T=100K, the bottom is at T=273 K
- The liquid is isothermal with $T=T_m$.
- The solid has initial $T(z)$ given by the Stefan solution (Turcotte and Schubert section 4.18)
$$
\theta = \frac{erf(\eta)}{erf(\lambda_1)}
$$
- Given the initial temperature, compute initial entropy S on the markers.
  

In [ ]:
# Importing (using/include) packages and files needed for the code to run
using SparseArrays
using LinearAlgebra
using IterativeSolvers
using WriteVTK
using Printf
using Statistics
using Dates
using SpecialFunctions
using Roots
using NLsolve
using Printf
using HDF5
include("Grid.jl")
include("Markers.jl")
include("Stokes.jl")
include("GridOperations.jl")
include("Topo.jl")
# Note: that we import pyplot last to avoid a name conflict with grid
using PyPlot
include("Visualization.jl")

# functions that relates entropy and temperature/pressure are define in TemperatureEntropy.jl file
include("TemperatureEntropy.jl")

In [ ]:
struct Materials
    alpha::Vector{Float64} # Thermal expansion (1/K)
    rho0::Vector{Float64} # Density (kg/m^3)
    Hr::Vector{Float64} # Radiogenic heat production (W/m^3)
    Cp::Vector{Float64} # Heat capacity (J/kg*K)
    kThermal::Vector{Float64} # Thermal conductivity (W/m*K)
    function Materials()
        new([0.0],[1e3],[0.0],[2.1e3],[2.2])
    end    
end

function update_marker_prop!(markers::Markers,materials::Materials)
    rho = markers.scalarFields["rho"]
    mmat = markers.integers[markers.integerFields["material"],:]
    for i in 1:markers.nmark
        markers.scalars[rho,i] = materials.rho0[mmat[i]]
    end
end

function update_marker_T_X!(markers::Markers,materials::Materials,options::Dict)
    T = markers.scalarFields["T"]
    X = markers.scalarFields["X"]
    S = markers.scalarFields["S"]    
    for i in 1:markers.nmark
        markers.scalars[T,i],markers.scalars[X,i] = compute_T_X_from_S((markers.scalars[S,i]),options)
    end
end

function initial_conditions!(markers::Markers,materials::Materials,options::Dict)
    material = markers.integerFields["material"]
    S = markers.scalarFields["S"]
    X = markers.scalarFields["X"]
    T = markers.scalarFields["T"]
    alpha = markers.scalarFields["alpha"]
    Cp = markers.scalarFields["Cp"]
    Hr = markers.scalarFields["Hr"]
    kThermal = markers.scalarFields["kThermal"]
    
    for i in 1:markers.nmark
        mx = markers.x[1,i]
        my = markers.x[2,i]
        markers.integers[material,i] = 1
        markers.scalars[alpha,i] = materials.alpha[1]
        markers.scalars[Cp,i] = materials.Cp[1]
        markers.scalars[Hr,i] = materials.Hr[1]
        markers.scalars[kThermal,i] = materials.kThermal[1]
        markers.scalars[T,i] = 200.0
        markers.scalars[X,i] = 0.0
        markers.scalars[S,i] = compute_S_from_T_X(markers.scalars[X,i],markers.scalars[T,i],options)
    end 
    # end loop over markers
    update_marker_prop!(markers,materials)
end

function get_interface(grid::CartesianGrid,mat::Matrix{Float64},contour_value::Float64)
    # Finding interfaces
    interface_position = zeros(Float64,grid.nx+1);
    for j in 1:grid.nx+1
        i = 1
        while i <= grid.ny
            if mat[i,j] == contour_value
                interface_position[j] = grid.yc[i]
                break
            elseif mat[i+1,j] < contour_value
                # interface is located within this cell.
                interface_position[j] = grid.yc[i] + (grid.yc[i+1]-grid.yc[i])/(mat[i+1,j]-mat[i,j])*(contour_value-mat[i,j])
                break
            end
            i = i+1
        end
    end
    return interface_position
end

In [ ]:
function model_run(options::Dict)
    W = 1e4
    H = 1e4
    ny = 50 
    nx = 51 #Int64(ceil(W/H*ny))
    gx = 0.0
    gy = 0.113
    
    # -1 = insulating, 1 = constant temp 
    Tbctype = [-1,-1,1,1] #left, right, top, bottom
    Tbcval = [0.0,0.0,100.0,200.0] #left, right, top, bottom
    markx = 6
    marky = 6
    seconds_in_year = 3.15e7
    plot_interval = 1e6*seconds_in_year # plot interval in seconds
    end_time = 3e7*seconds_in_year
    dtmax = plot_interval
    
    grid = CartesianGrid(W,H,nx,ny)
    println("Grid resolution(ny x nx) : $ny x $nx")
    println("Cell size in the x-direction is $(grid.W/(grid.nx-1))")
    println("Cell size in the y-direction is $(grid.H/(grid.ny-1))")
    
    materials = Materials()
    println("Creating Markers...")
    @time markers = Markers(grid,["alpha","T","rho","Cp","Hr","kThermal","S","X"],["material"] ; nmx=markx,nmy=marky,random=false)
    println("Initial condition...")
    @time initial_conditions!(markers,materials,options)

    ### Setting up agruments for termination criteria ###
    max_step::Int64=200
    max_time::Float64=-1
    max_time = max_time == -1.0 ? typemax(Float64) : max_time
    max_step = max_step == -1 ? typemax(Int64) : max_step

    time = 0.0
    iout= 0
    last_plot = 0.0
    dt = dtmax
    
    rho_c = nothing
    kThermal_vx = nothing
    kThermal_vy = nothing
    Hr = nothing
    kThermal = nothing
    dTmax = nothing
    dTemp = nothing
    Tnew = nothing
    Tlast = nothing
    Snew = nothing
    Slast = nothing
    Xnew = nothing
    Xlast = nothing
    q_vx = nothing
    q_vy = nothing
    
    itime = 1 
    output_dir = "debugging"
    terminate = false
    while !terminate
        
        update_marker_prop!(markers,materials)
        rho_c_new,Hr_new,kThermal_new,Slast_new,Xlast_new, = marker_to_stag(markers,grid,["rho","Hr","kThermal","S","X"],"center")
        rhocp = markers.scalars[markers.scalarFields["rho"],:] .* markers.scalars[markers.scalarFields["Cp"],:]
        Tlast_new, = marker_to_stag(markers,grid,["T"],"center",extra_weight = rhocp)
        kThermal_vx_new, = marker_to_stag(markers,grid,["kThermal",],"vx")
        kThermal_vy_new, = marker_to_stag(markers,grid,["kThermal",],"vy")
        
        # T_solution = zeros(grid.ny,grid.nx)
        # for j in 1:grid.nx
        #     for i in 1:grid.ny
        #         n = grid.yc[i] / (2*sqrt(options["thermal diffusivity"]*time))
        #         T_solution[i,j] = erf(n)*(200.0-100.0) + (100.0)
        #     end
        # end
    
        # deal with any NaN values from interpolation:
        if itime > 1
            replace_nan!(rho_c,rho_c_new)
            replace_nan!(kThermal_vx,kThermal_vx_new)
            replace_nan!(kThermal_vy,kThermal_vy_new)
            replace_nan!(Hr,Hr_new)
            replace_nan!(kThermal,kThermal_new)
            replace_nan!(Tlast,Tlast_new)
            replace_nan!(Slast,Slast_new)
            replace_nan!(Xlast,Xlast_new)
        end
        # Copy field data
        rho_c = copy(rho_c_new)
        kThermal_vx = copy(kThermal_vx_new)
        kThermal_vy = copy(kThermal_vy_new)
        Hr = copy(Hr_new)
        kThermal = copy(kThermal_new)
        Tlast = copy(Tlast_new)    
        Slast = copy(Slast_new)
        Xlast = copy(Xlast_new)

        # Initial guess for S
        Snew = copy(Slast)
        # Initial guess for T
        Tnew,Xnew = update_T_X_from_S(Snew,options)
        Tnew,Xnew,Snew = ghost_nodes_center_TXS(grid,Tnew,Xnew,Slast,Tbctype,Tbcval,options)

        diffusion_timestep = calculate_diffusion_timestep(grid,options)
        if itime > 1
            this_dtmax = min(1.2*dt,dtmax)
        else
            this_dtmax = dtmax
        end
        if this_dtmax > diffusion_timestep
            dt = diffusion_timestep
        else 
            dt = this_dtmax
        end    
            
        last_T_norm = NaN
        T_norm = NaN
        dT = nothing
        dTmax = Inf
        dS = nothing
        dSmax = Inf
        tolerance = 1e-8
        dTnorm = []
        ititer = []
        titer = 1 
        max_titer = 300
        for titer=1:max_titer
                        
            # Computing conductive heat flux
            q_vx,q_vy = compute_q_cond(grid,Tnew,kThermal_vx,kThermal_vy) # W/m^2
            
            # Computing the new entropy
            Snew = compute_S_new(grid,Tnew,rho_c,Hr,q_vx,q_vy,Slast,dt);
    
            # Updating the new temperature and new melt fraction from the new entropy
            Tnew,Xnew = update_T_X_from_S(Snew,options)             
            Tnew,Xnew,Snew = ghost_nodes_center_TXS(grid,Tnew,Xnew,Snew,Tbctype,Tbcval,options)  
       
            if titer > 1
                last_T_norm = T_norm;
                T_norm = norm(Tnew[2:grid.ny,2:grid.nx]);
                push!(dTnorm,abs(T_norm-last_T_norm))
                push!(ititer,titer)
            else
                T_norm = norm(Tnew[2:grid.ny,2:grid.nx]);
                push!(dTnorm,abs(T_norm-last_T_norm))
                push!(ititer,titer)
            end
            
            # Computing the maximum temperature change
            dT = Tnew - Tlast
            dTmax = maximum(abs.(dT[2:grid.ny,2:grid.nx]))

            # Computing the maximum entropy change
            dS = Snew - Slast
            dSmax = maximum(abs.(dS[2:grid.ny,2:grid.nx]))

            # Checking for convergence:
            if titer > 1 && abs(T_norm-last_T_norm) < tolerance
                # println("Converged after $titer iterations.")
                break
            elseif titer == max_titer
                terminate = true
                @error("Did not converged")
            elseif any(isnan.(dT))
                terminate = true
                @error("NaN or Inf apperred")
            end
        end

        # Updating entropy on the markers by projecting dS from the cell centers to the markers
        cell_center_change_to_markers!(markers,grid,dS,"S")
        update_marker_T_X!(markers,materials,options)

        # Checking Termination Criteria, time is in Myr, amplitude is in meters
        if time >= max_time || itime >= max_step 
            terminate = true
            figure()
            plot(Tnew[:,1],grid.yc/1e3,"r--",label="Temperature")
            gca().set_ylabel("Depth(km)")
            gca().set_xlabel("Temperature(K)")
            gca().invert_yaxis()
            show()
            
            figure()
            title("Temperature Profile at timestep $itime")
            pcolor(grid.xc/1e3,grid.yc/1e3,Tnew)
            colorbar(cmap="viridis",label="K")
            gca().set_ylabel("Depth(km)")
            gca().set_xlabel("Width(km)")
            gca().invert_yaxis()
            show()
        end
        
        # figure()
        # title("Numerical vs. Analytical at timestep $itime")
        # plot(T_solution[:,1],"b-",label="Analytical")
        # plot(Tnew[:,1],"r--",label="Numerical")
        # # gca().set_ylabel("Depth(km)")
        # # gca().set_xlabel("Temperature(K)")
        # legend()
        # gca().invert_yaxis()
        # show()
        
        # figure()
        # title("At timestep = $itime")
        # plot(ititer,dTnorm,"-")
        # gca().set_ylabel(L"dTnorm")
        # gca().set_xlabel(L"Iterations")
        # show()

        # if time == 0.0 || mod(itime,10) == 0 || terminate
        #     last_plot = time 
        #     # Markers output
        #     name1 = @sprintf("%s/markers.%04d.vtp",output_dir,iout)
        #     println("Writing visualization file = ",name1)
        #     visualization(markers,time/seconds_in_year;filename=name1)
        #     iout += 1
        # end
        
        # Checking Termination Criteria, time is in Myr, amplitude is in meters
        if time >= max_time || itime >= max_step     
            terminate = true
        end
        
        time += dt
        itime += 1
    end
    return grid,itime
end

In [ ]:
options = Dict()
options["latent heat of fusion"] = 3.34e5 #J/kg
options["specific heat of ice"] = 2.1e3 # J/kg*K (ice)
options["density of ice"] = 1e3 # kg/m^3
options["thermal conductivity of ice"] = 2.2 # W/m*K
options["thermal diffusivity"] = options["thermal conductivity of ice"] / (options["density of ice"]*options["specific heat of ice"]) # m^2/s
options["Tm"] = 273.0 # K
options["To"] = 100.0 # K
grid,itime = model_run(options);